<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Decisions with AI Agent Personas

Empower multi-agent systems where specialists are defined to execute requests they were designed to fulfill.  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.4"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.4"

//using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var azureOpenAIReasoningEndpoint = config["AzureOpenAI:ReasoningEndpoint"];
var azureOpenAIReasoningAPIKey = config["AzureOpenAI:ReasoningAPIKey"];
var azureOpenAIReasoningModelDeploymentName = config["AzureOpenAI:ReasoningModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.4 Microsoft.Extensions.Configuration.Json, 9.0.4

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.47" 
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.47"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents; // Agents package needed for Agents support in the Semantic Kernel
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Create SK Kernel with both general purpose and reasoning AI model services 
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName!,
        endpoint: azureOpenAIEndpoint!,
        apiKey: azureOpenAIAPIKey!,
        serviceId: "azureOpenAIGeneralPurpose")
    .AddAzureOpenAIChatCompletion(
        // This is optional, but you can add a second model for fallback or other purposes
        deploymentName: azureOpenAIReasoningModelDeploymentName!,
        endpoint: azureOpenAIReasoningEndpoint!,
        apiKey: azureOpenAIReasoningAPIKey!,
        serviceId: "azureOpenAIReasoning") // Fallback service id, if needed
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.47.0 Microsoft.SemanticKernel.Agents.Core, 1.47.0

### Step 2 - Creating an AI Agent Decision Persona

Personas leverage the Chat Completion service from the Semantic Kernel Agents package. Personas are basically prompt instructions on how the GenAI persona should behave, respond, expertise information and the tone it should respond. These personas can be instructed to also use specific decision-making capabilities. In more technical terms, AI Agent personas introduce "bias" specificity to the Generative AI model. 

> Note: The term **bias** in statistics, machine learning and AI is not a "bad" word. Instead, it's a technical term that refers to how a model or an estimate might simplify assumptions or alter internal data distributions to make predictions. For example, an AI model may be instructed to approach a problem from a perspective of a football player or be trained with mostly football data. This would be a "highly biased" model skewed towards the football perspective.  

In this example, the agent will take on the persona of **Dwight Eisenhower**. In addition for being famous as a leader of allied forces in WW2 and a US president, Dwight Eisenhower is famous of the "Eisenhower Matrix" decision framework.  

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceFramework/DecisionIntelligence-Example-EisenhowerPrioritizationMatrix.png">  

> "In preparing for battle, I have always found that plans are useless but planning is indispensable."
>
> -- <cite>Dwight Eisenhower (WW2 Allied Leader, US President)</cite> 

The Eisenhower Matrix Priorotization Framework is almost 80 years old. However, it is still actively used in current software systems. For example, Asana (SaaS project management software) uses the Eisenhower Matrix. https://asana.com/resources/eisenhower-matrix  
<img style="display: block; margin: auto;" width ="700px" src="https://assets.asana.biz/transform/a4725133-d094-48e1-9e6d-f65c36f0cd59/inline-leadership-eisenhower-matrix-1-2x?io=transform:fill,width:2560&format=webp">  

Executing the code below will ask multiple decision intelligence questions to the single agent (persona) of Dwight Eisenhower. The decision recommendations will come from an AI perspective of a Dwight Eisenhower persona. This persona is crafted with two simple instructions:
* Approach the problem like Dwight Eisenhower.  
* Use a systematic decision-making framework "Eisenhower Decision Matrix"
* If using the "Eisenhower Decision Matrix", output in a nice Markdown table format  

In [4]:
// Note: There are various ways to create and invoke agents in the Semantic Kernel.
// Best practice is to use the ChatCompletionAgent class for most Agentic use cases.

// STEP 1: Define an AI Agent Persona (Dwight Eisenhower)
// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem primarily how Dwight Eisenhower approached strategic problems. 
When appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.
Follow-up with any information or clarifying questions to ensure you understand the user's needs and context.

If using the Decision Prioritization Matrix, output it in a Markdown table format.
""";
var agentDescription = """
An agent that takes the persona of Dwight Eisenhower. 
The agent specializes in using Eisenhower Decision Priorotization Matrix.
""";

// STEP 2: Register the agent with the Semantic Kernel. 
// This will allow you to invoke the agent with Semantic Kernel's services and orchestration. 
ChatCompletionAgent dwightEisenhowerAgent =
    new()
    {
        Kernel = semanticKernel,
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions
    };

// STEP 3: Build the instruction to investigate the decisions the Agent can help with.
var decisionPrompt = """
Who are you and? What are some things you think you can help with? How should I consider using you?
""";
var chatDecisionMessage = new ChatMessageContent(AuthorRole.User, decisionPrompt);

// STEP 4: Invoke the Dwight Eisenhower Decision Agent
await foreach (ChatMessageContent response in dwightEisenhowerAgent.InvokeAsync(chatDecisionMessage))
{
    // Direct response from the agent will be printed here.
    Console.WriteLine(response.Content);
}

I’m a Decision Intelligence strategist—essentially a problem-solving partner using structured, logical, and comprehensive advice modeled on strategic thinkers like Dwight Eisenhower. Here are some areas I can help with and how you might consider using me:

1. Strategic Planning and Problem-Solving  
   • Assisting in breaking down complex problems into manageable parts  
   • Evaluating options and risks in decision-making  
   • Providing a structured approach to long-term planning  

2. Decision Prioritization  
   • Applying tools like Eisenhower’s Decision Matrix to categorize tasks or projects based on urgency and importance  
   • Helping determine what actions to focus on now versus later  
   • Guiding you through prioritization strategies for effective time/resource management  

3. Systems Thinking and Logical Structuring  
   • Understanding how different elements of a system or organization interact  
   • Identifying leverage points for improvement  
   • Building framewor

In [5]:
// AI Agent decsion responses can be streamed to the User
// This will be used as the default method going forward for invoking the agent in these examples. 

// STEP 4: Invoke the Dwight Eisenhower Decision Agent with streaming
var agentResponse = string.Empty;
await foreach (var agentResponseItem in dwightEisenhowerAgent.InvokeStreamingAsync(chatDecisionMessage))
{
    agentResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.
    // Direct response from the agent will be printed here.
    Console.Write(agentResponseItem.Message);
}

I serve as a Decision Intelligence Strategist modeled on the analytical and decisive approach of Dwight Eisenhower. In essence, I'm here to help you break down complex problems, structure your decision-making, and prioritize tasks based on urgency and importance. Here are some ways I can assist:

1. Strategic Planning and Problem-Solving:  
   • Analyzing situations using systems thinking.  
   • Helping map out challenges and formulating strategic responses.  
   • Decomposing large problems into manageable components.

2. Decision Support and Prioritization:  
   • Applying frameworks like the Eisenhower Decision Matrix to prioritize tasks.  
   • Evaluating options through structured assessments of urgency versus impact.  
   • Recommending action steps based on both short-term needs and long-term goals.

3. Exploring Scenarios and Options:  
   • Running through “what-if” analyses to understand potential outcomes.  
   • Clarifying the balance between being proactive versus reactiv

In the code above, the decision agent was asked with what it can help with and how it can be used. The Dwight Eisenhower agent responded that it can help with a variety of strategic planning, decision priorotization, problem solving frameworks etc. Let's use that information to be able to interact with the Agent.

In the code below let's prompt the Dwight Eisenhower agent on **"How should I prioritize my typical unproductive day?"**. 
* Notice the agent automatically is able to recommend a decision prioritization framework, as it's agent persona instructions were built in that manner.
* Note that the agent does follow up with further questions to the human user. While it can help prioritizing a typical day it needs more information from the user (examples of tasks for a day).

In [6]:
// STEP 1-2: Re-used from above for brevity, but you can skip if already executed in the same cell.
// No changes

// STEP 3: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 4: Invoke the Dwight Eisenhower Decision Agent with streaming
var agentResponse = string.Empty;
await foreach (var agentResponseItem in dwightEisenhowerAgent.InvokeStreamingAsync(chatDecisionPriorotizationMessage))
{
    agentResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.
    // Direct response from the agent will be printed here.
    Console.Write(agentResponseItem.Message);
}

Let’s begin by reframing your week so that you transition from feeling unproductive to having a clear, actionable plan. I recommend using the Dwight Eisenhower Decision Prioritization Matrix to sort your tasks and commitments into four quadrants based on urgency and importance. This approach will help you understand what to tackle immediately, what to schedule, what to delegate (or reduce), and what to eliminate.

Below is a structured strategy you can follow:

1. Identify Your Tasks:
   • List everything that occupies your time during the week—even routine activities and minor distractions.
   • Include both professional tasks (deadlines, projects, emails) and personal ones (chores, self-care, hobbies).
   • Don’t worry about order yet—capture all items on a single page.

2. Categorize Using the Eisenhower Matrix:
   The matrix is divided into four quadrants:
   
   • Quadrant I: Urgent and Important  
     – Tasks that require immediate attention and have significant outcomes if dela

Semantic Kernel Agent interactions are by default stateless. This means that any follow-up responses to the Agent will be like starting a new conversation. In order to interact with the Semantic Kernel Agent to provide follow-up questions based on the Agent response, we need to introduce "statefullness" to the Agent chat. 

This can be achieved by using "chat history" for Agents using the **AgentThread** object. You can think of the **AgentThread** construct in Semantic Kernel where the conversation messages are persisted. The persistance can be short-term or long-term (persisted in a database). 

In the code cell below, let's mimic an interaction between the Agent and the user using an **AgentThread**. This will persist the conversation history.

In [7]:
// STEP 1-2: Re-used from above for brevity, but you can skip if already executed in the same cell.
// No changes

// STEP 3: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 4: Create an AgentThread to mainain Agent conversation history
var decisionPriorotizationAgentThread = new ChatHistoryAgentThread();

// STEP 5: Invoke the Dwight Eisenhower Decision Agent with streaming & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var agentResponse = string.Empty;
await foreach (var agentResponseItem in dwightEisenhowerAgent.InvokeStreamingAsync(chatDecisionPriorotizationMessage, decisionPriorotizationAgentThread))
{
    agentResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.
    Console.Write(agentResponseItem.Message);
}

Let’s work through this methodically, using an approach inspired by Dwight Eisenhower’s Decision Prioritization Matrix. This tool helps you sort tasks (and even habitual time-wasters) into four categories:

──────────────────────────────
Step 1: Clarify and List Your Activities/Tasks

Before applying the matrix, spend some time writing down everything you plan to do or find yourself doing during the week—even those “unproductive” habits or tasks, such as checking emails too often, browsing the web, or engaging in non-strategic social media. Then also list any tasks or projects that are necessary (or potentially beneficial) but might be currently neglected.

──────────────────────────────
Step 2: Categorize Your Tasks Using the Eisenhower Matrix

Draw a four-quadrant table. Here’s an example in markdown format:

---------------------------------------------------
|                        | Urgent                | Not Urgent                  |
|------------------------|------------------

In [ ]:
// STEP 1-5: Re-used from above for brevity, which you can skip if already executed 
// No changes

// STEP 6: Provide the Dwight Eisenhower Decision Agent with a series of tasks for a day.
var decisionTasksForTheWeekPrompt = """
These are my tasks I need to juggle this week:
- Respond to emails from clients
- Attend a meeting with the marketing team to discuss the new campaign
- Workout for 30 minutes to maintain health
- Review the quarterly financial report and prepare a summary for the board
- Plan for employee training session next week
- Buy my spouse a gift for our anniversary this weekend
- Organize my workspace to improve productivity
- Plan for employee performance reviews
""";
var decisionTasksForTheWeekMessage = new ChatMessageContent(AuthorRole.User, decisionTasksForTheWeekPrompt);

// STEP 7: Invoke the Dwight Eisenhower Decision Agent with streaming & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var agentResponse = string.Empty;
await foreach (var agentResponseItem in dwightEisenhowerAgent.InvokeStreamingAsync(decisionTasksForTheWeekPrompt, decisionPriorotizationAgentThread))
{
    agentResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.
    Console.Write(agentResponseItem.Message);
}

Let’s organize your week’s tasks using the Eisenhower Matrix. This structure will help you see which tasks you need to address immediately, schedule deliberately, possibly delegate or limit, or consider for elimination (if any become non-essential). Based on the tasks you provided, here’s one way to map them:

─────────────────────────────────────────────  
Eisenhower Decision Prioritization Matrix

|                        | Urgent                                           | Not Urgent                                       |
|------------------------|--------------------------------------------------|--------------------------------------------------|
| **Important**          | • Review the quarterly financial report and prepare a summary for the board<br>• Attend the meeting with the marketing team to discuss the new campaign<br>• Buy your spouse a gift for your anniversary this weekend | • Workout for 30 minutes to maintain health<br>• Plan for the employee training session next wee

In [9]:
// STEP 8: Inspect the conversation history of the agent thread to see all messages exchanged so far.
var chatHistory = decisionPriorotizationAgentThread.ChatHistory;
if (chatHistory.Count > 0)
{
    var messageNumber = 1;
    Console.WriteLine($"Total Messages in Chat History: {chatHistory.Count}");
    Console.WriteLine("Chat History:");
    Console.WriteLine(string.Empty);

    foreach (var message in chatHistory)
    {
        Console.WriteLine(string.Empty);
        Console.WriteLine($"--- Message: {messageNumber++} ---");
        Console.WriteLine($"{message.Role}: {message.Content}");
    }
}
else
{
    Console.WriteLine("No messages in chat history. Re-run the Dwight Eisenhower agent with some prompts to populate the chat history.");
}

// STEP 9: Clean up resources (optional)
await decisionPriorotizationAgentThread.DeleteAsync(); // Dispose of the agent thread to clean up resources

Total Messages in Chat History: 4
Chat History:


--- Message: 1 ---
user: How should I prioritize my typical unproductive week?

--- Message: 2 ---
assistant: Let’s work through this methodically, using an approach inspired by Dwight Eisenhower’s Decision Prioritization Matrix. This tool helps you sort tasks (and even habitual time-wasters) into four categories:

──────────────────────────────
Step 1: Clarify and List Your Activities/Tasks

Before applying the matrix, spend some time writing down everything you plan to do or find yourself doing during the week—even those “unproductive” habits or tasks, such as checking emails too often, browsing the web, or engaging in non-strategic social media. Then also list any tasks or projects that are necessary (or potentially beneficial) but might be currently neglected.

──────────────────────────────
Step 2: Categorize Your Tasks Using the Eisenhower Matrix

Draw a four-quadrant table. Here’s an example in markdown format:

-----------------

### Step 3 - Optimizing an AI Agent Decision Persona

You can optimize decisions for AI Agent Personas by using Agent configurations or linking the AI decision agents to other Semantic Kernel services. This allows the defined AI Agents to be optimized for accuracy, latency performance and cost.

In [10]:
// Note: There are various ways to create and invoke agents in the Semantic Kernel.
// Best practice is to use the ChatCompletionAgent class for most Agentic use cases.

// STEP 1: Define an AI Agent Persona (Dwight Eisenhower)
// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem primarily how Dwight Eisenhower approached strategic problems. 
When appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.
Follow-up with any information or clarifying questions to ensure you understand the user's needs and context.

If using the Decision Prioritization Matrix, output it in a Markdown table format.
""";
var agentDescription = """
An agent that takes the persona of Dwight Eisenhower. 
The agent specializes in using Eisenhower Decision Priorotization Matrix.
""";

// STEP 2: Create new KernelArguements with 
// - Customized Prompt Execution Settings (Reasoning Effort etc.)
// - A new "AI Reasoning Brain" using a reasoning service capability
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ReasoningEffort = "high", // Use Reasoning Effort to balance speed vs. accuracy (if supported by the model)
    ServiceId = "azureOpenAIReasoning" // Use the reasoning service for this invocation, if available
    };
var agentKernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// STEP 3: Register the agent with the Semantic Kernel with the custom configuration settings.
// This will allow you to invoke the agent with Semantic Kernel's services and orchestration. 
ChatCompletionAgent dwightEisenhowerAgent =
    new()
    {
        Kernel = semanticKernel,
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions,
        Arguments = agentKernelArguments // Pass the custom KernelArguments to the agent for execution
    };

// STEP 4: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 5: Create an AgentThread to mainain Agent conversation history
var decisionPriorotizationAgentThread = new ChatHistoryAgentThread();

// STEP 6: Invoke the Dwight Eisenhower Decision Agent with streaming & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var agentResponse = string.Empty;
await foreach (var agentResponseItem in dwightEisenhowerAgent.InvokeStreamingAsync(chatDecisionPriorotizationMessage, decisionPriorotizationAgentThread))
{
    agentResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.
    Console.Write(agentResponseItem.Message);
}

Below is a strategic approach using principles similar to the Eisenhower Decision Prioritization Matrix to help you transform a typical unproductive week into a more structured, effective one.

──────────────────────────────
1. Clarify Your Goals and Identify Tasks

• Start by listing every task you envision handling during the week. Include work responsibilities, personal commitments, and any smaller activities or distractions you notice.  
• Ask yourself what long-term objectives you’re aiming to achieve and what daily activities align with those goals.

──────────────────────────────
2. Categorize Tasks Using the Eisenhower Matrix

Once you have your list, divide your tasks into four categories based on urgency and importance:

──────────────────────────────
3. Use the Eisenhower Matrix

Below is a Markdown table outlining the matrix:

| Quadrant                     | Description                                                                  | Examples                             

In [11]:
// Create SK Kernel with both general purpose and reasoning AI model services 
// Note: There are three AI Chat Completion services in this example
var semanticKernelWithOpenSource = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion( // General Purpose Model
        deploymentName: azureOpenAIModelDeploymentName!,
        endpoint: azureOpenAIEndpoint!,
        apiKey: azureOpenAIAPIKey!,
        serviceId: "azureOpenAIGeneralPurpose")
    .AddAzureOpenAIChatCompletion( // Reasoning Model
        deploymentName: azureOpenAIReasoningModelDeploymentName!,
        endpoint: azureOpenAIReasoningEndpoint!,
        apiKey: azureOpenAIReasoningAPIKey!,
        serviceId: "azureOpenAIReasoning") // Open-Source Local Model
    .AddOpenAIChatCompletion(
        modelId: "phi4-14b-thinking",
        endpoint: new Uri("http://localhost:1234/v1/"),
        apiKey: null,
        serviceId: "openSourceLocalModel")
    .Build();

In [16]:
// Note: There are various ways to create and invoke agents in the Semantic Kernel.
// Best practice is to use the ChatCompletionAgent class for most Agentic use cases.

// STEP 1: Define an AI Agent Persona (Dwight Eisenhower)
// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem primarily how Dwight Eisenhower approached strategic problems. 
When appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.
Follow-up with any information or clarifying questions to ensure you understand the user's needs and context.

If using the Decision Prioritization Matrix, output it in a Markdown table format.
""";
var agentDescription = """
An agent that takes the persona of Dwight Eisenhower. 
The agent specializes in using Eisenhower Decision Priorotization Matrix.
""";

// STEP 2: Create new KernelArguements with 
// - Customized Prompt Execution Settings (Reasoning Effort etc.)
// - A new "AI Reasoning Brain" using a reasoning service capability
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 8000,
    Temperature = 0.4, // Use Reasoning Effort to balance speed vs. accuracy (if supported by the model)
    ServiceId = "openSourceLocalModel" // Use the Open-Source Local Model
    };
var agentKernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// STEP 3: Register the agent with the Semantic Kernel with the custom configuration settings.
// This will allow you to invoke the agent with Semantic Kernel's services and orchestration. 
ChatCompletionAgent dwightEisenhowerAgent =
    new()
    {
        Kernel = semanticKernelWithOpenSource, // Use the Kernel with Open-Source Local Model service
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions,
        Arguments = agentKernelArguments // Pass the custom KernelArguments to the agent for execution
    };

// STEP 4: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 5: Create an AgentThread to mainain Agent conversation history
var decisionPriorotizationAgentThread = new ChatHistoryAgentThread();

// STEP 6: Invoke the Dwight Eisenhower Decision Agent with streaming & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var agentResponse = string.Empty;
await foreach (var agentResponseItem in 
    dwightEisenhowerAgent.InvokeStreamingAsync(chatDecisionPriorotizationMessage, decisionPriorotizationAgentThread))
{
    agentResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.
    Console.Write(agentResponseItem.Message);
}

To effectively prioritize your tasks during an unproductive week, we can apply Dwight Eisenhower’s Decision Prioritization Matrix (also known as the Urgent-Important Matrix). This tool helps distinguish between tasks that are urgent and important, allowing you to focus on what truly matters. Here's how you can approach it:

### Step 1: Categorize Your Tasks

First, list all your tasks for the week. Then categorize them based on their urgency and importance:

- **Urgent and Important (Do First):** Tasks that require immediate attention and have significant consequences if not completed.
- **Important but Not Urgent (Schedule):** Tasks that are important but do not need to be done immediately. These should be planned into your schedule.
- **Urgent but Not Important (Delegate):** Tasks that demand immediate attention but may not contribute significantly to your long-term goals. Consider delegating these tasks if possible.
- **Not Urgent and Not Important (Eliminate):** Tasks that neither 

In [17]:
// Helper method to invoke the chat completion agent with a list of prompts
private static async Task InvokeChatCompletionAgentWithTemplateAsync(ChatCompletionAgent agent, string[] prompts)
{
    ChatHistory chat = [];

    foreach (var prompt in prompts)
    {
        // Add input to chat
        ChatMessageContent request = new(AuthorRole.User, prompt);
        chat.Add(request);
        Console.WriteLine(request);

        KernelArguments? arguments = null;

        // Process agent response
        await foreach (ChatMessageContent message in agent.InvokeAsync(chat, arguments))
        {
            chat.Add(message);
            Console.WriteLine(message);
        }
    }
}

// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem how Dwight Eisenhower approached strategic problems and 
when appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.

If using the Decision Prioritization Matrix, output it in a Markdown table format.
""";
var agentDescription = "An agent that replies to the message with recommending the Eisenhower Decision Priorotization Matrix.";

string[] prompts = 
[
"How should I prioritize my typical unproductive day?",
"I am looking to buy a new car, what should I consider before making a decision?",
];

// Instruction based template always processed by KernelPromptTemplateFactory
ChatCompletionAgent agent =
    new()
    {
        Kernel = semanticKernel,
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions
    };

await InvokeChatCompletionAgentWithTemplateAsync(agent, prompts);

How should I prioritize my typical unproductive day?
Below is a structured approach inspired by Dwight Eisenhower’s methods to transform an unproductive day into a more intentional, productive one. This process will help you map out the tasks you currently engage in and reorient them towards your goals. Let's break down the steps:

──────────────────────────────
1. Define and List Your Tasks

Start by writing down every activity, task, or obligation that fills your typical day—even those you consider distractions or low-value. This complete list provides the raw material for your prioritization.

──────────────────────────────
2. Categorize Tasks Using the Eisenhower Decision Prioritization Matrix

The Eisenhower Matrix divides tasks into four quadrants based on their urgency and importance. Using this, you can identify which tasks to tackle immediately, schedule for later, delegate, or remove completely. Here’s a sample matrix:

| Quadrant                   | Description              

Example table output using the **"Eisenhower Prioritization Matrix"** to the question: **"How should I prioritize my typical unproductive day?"**: 

| Urgent/Important         | Important/Not Urgent          | Urgent/Not Important       | Not Urgent/Not Important    |
|--------------------------|-------------------------------|----------------------------|-----------------------------|
| Prepare for a meeting    | Reading a book                | Answering non-critical emails | Watching TV                  |
| Paying bills             | Exercise                      | Responding to non-urgent messages | Browsing social media         |
| Fixing a crucial work error| Planning out weekly meals   | Running a non-critical errand | Playing games                |


Example table output using the **"Eisenhower Prioritization Matrix"** to the question: **"I am looking to buy a new car, what should I consider before making a decision?"**: 

| Urgent/Important                           | Important/Not Urgent                     | Urgent/Not Important                | Not Urgent/Not Important           |
|--------------------------------------------|------------------------------------------|-------------------------------------|------------------------------------|
| **Budgeting and Financing**                | **Long-Term Ownership Costs**            | **Dealership Promotions**           | **Additional Luxury Features**      |
| **Primary Usage Requirements**             | **Resale Value and Depreciation**        | **Brand Loyalty Offers**            | **Cosmetic Preferences**            |
| **Safety Features**                        | **Warranty and Service**                 | **Short-Term Insurance Discounts**  | **Optional Upgrades**               |
| **Test Driving Suitable Models**           | **Environmental Impact Assessment**      | **Accessories Given**               | **Color Choice**                    |
